In [1]:
import numpy as np
import torch.nn as nn

import random
import torch

from tqdm import tqdm
from data import*

In [2]:
# We define an architecture where we introduce a Quantum State the model retuns an escalar that tell us if the state is entangled or not

class EntanglementNN(nn.Module):
    def __init__(self):
        super(EntanglementNN, self).__init__()

        # Convolutional layers to extract features
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=4, kernel_size=3, padding=1)   # Output: 4 x 4 x 4
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, padding=1)   # Output: 8 x 4 x 4
        self.conv3 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)  # Output: 16 x 4 x 4

        # Batch Normalization layers for better convergence
        self.bn1 = nn.BatchNorm2d(4)
        self.bn2 = nn.BatchNorm2d(8)
        self.bn3 = nn.BatchNorm2d(16)

        # Fully connected layers to process the flattened features
        self.fc1 = nn.Linear(16 * 4 * 4, 32)  # First fully connected layer
        self.fc2 = nn.Linear(32, 1)           # Final output layer (scalar)

        # Dropout layer to reduce overfitting
        self.dropout = nn.Dropout(0.5)

        # Activation functions
        self.relu = nn.ReLU()        # ReLU activation for hidden layers
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation for output (binary classification)
    
    def forward(self, x):
        # Forward pass through the convolutional layers with BatchNorm and ReLU activation
        x = self.relu(self.bn1(self.conv1(x)))  # Output: 4 x 4 x 4
        x = self.relu(self.bn2(self.conv2(x)))  # Output: 8 x 4 x 4
        x = self.relu(self.bn3(self.conv3(x)))  # Output: 16 x 4 x 4

        # Flatten the feature maps before passing into the fully connected layers
        x = x.view(x.size(0), -1)  # Output: (batch_size, 16 * 4 * 4)

        # Forward pass through the fully connected layers
        x = self.dropout(self.relu(self.fc1(x)))  # Output: 32
        x = self.fc2(x)                           # Output: 1 (scalar)

        # Apply the sigmoid activation to get the probability output
        x = self.sigmoid(x)

        return x

# Instantiate the model
model = EntanglementNN()


In [3]:
# We load a previously trained model using Entanglement_CNN_train.ipynb

trained_model = torch.load('model.pth')
model.load_state_dict(trained_model)

<All keys matched successfully>

In [44]:
# For test our model we randomly generates a bunch of 100000 states and, with the Peres criterion, we classify the states in entangled
# and separable. Then, we use the model for the same task and then we verify if the prediction made for the model is correct or not.

N_test      = 100000
state, p    = create_dataset(N_test, criterion="ppt")

test_tick   = []
check_model = torch.abs(model(state) - p)
check_model = check_model.detach().numpy()
check_model = check_model.flatten()

checked     = check_model[ check_model > 0.01 ]
wrong_cases = checked.shape[0]

success_ratio = 1 - wrong_cases/len(p)

print("The success ratio of our model is ", success_ratio*100,"%")

Generating states with Peres criterion: 100%|██████████| 100000/100000 [00:41<00:00, 2418.43it/s]


The success ratio of our model is  64.41900000000001 %
